In [2]:
import yfinance as yf
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "iframe_connected"  # or "iframe_connected"

# Download S&P 500 data
sp500 = yf.download("^GSPC", start="2010-01-01", end="2025-01-01")

# Extract Year
sp500['Year'] = sp500.index.year

# Group by Year and take the mean
sp500_yearly = sp500.groupby('Year').mean(numeric_only=True)

# ✅ Flatten multi-level column names
sp500_yearly.columns = [col if not isinstance(col, tuple) else col[0] for col in sp500_yearly.columns]

# Now bring Year back as a column
sp500_yearly = sp500_yearly.reset_index()

# ✅ Rename for clarity
sp500_yearly = sp500_yearly.rename(columns={'Close': 'AvgClose'})

# Round for nicer display
sp500_yearly['AvgClose'] = sp500_yearly['AvgClose'].round(2)

# ✅ Debug: See final dataframe
print(sp500_yearly.head())
print(sp500_yearly.columns)

# ✅ Clean Plotly Express chart
fig = px.line(
    sp500_yearly,
    x="Year",
    y="AvgClose",
    title="📈 S&P 500 Yearly Average Close (2010–2024)",
    markers=True
)

fig.update_traces(line=dict(width=3, color="#1f77b4"), marker=dict(size=8))
fig.update_layout(
    plot_bgcolor="white",
    hovermode="x unified",
    xaxis=dict(
        showgrid=True,
        gridcolor="lightgray",
        tickmode="linear"
    ),
    yaxis=dict(
        title="Average Close Price (USD)",
        showgrid=True,
        gridcolor="lightgray"
    )
)

fig.show()


C:\Users\Liam Sydney\AppData\Local\Temp\ipykernel_17208\2816719890.py:8: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed


   Year  AvgClose         High          Low         Open        Volume
0  2010   1139.97  1146.573372  1131.280990  1139.369761  4.569371e+09
1  2011   1267.64  1277.223450  1257.176747  1267.618211  4.109056e+09
2  2012   1379.35  1385.845401  1371.269002  1378.680638  3.631677e+09
3  2013   1643.80  1649.615712  1635.740156  1642.298649  3.358967e+09
4  2014   1931.38  1938.835599  1922.292981  1930.754485  3.354783e+09
Index(['Year', 'AvgClose', 'High', 'Low', 'Open', 'Volume'], dtype='object')


In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("leonardopena/top-spotify-songs-from-20102019-by-year")

print("Path to dataset files:", path)

C:\Users\Liam Sydney\Desktop\tensorflow_project\tf-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html



Path to dataset files: C:\Users\Liam Sydney\.cache\kagglehub\datasets\leonardopena\top-spotify-songs-from-20102019-by-year\versions\1


In [4]:
import os
import pandas as pd

# Path to the CSV inside the downloaded folder
csv_path= os.path.join(path, "top10s.csv")

# Load it
df_spotify = pd.read_csv(csv_path, encoding="latin1")
print(df_spotify.head())
print(df_spotify.columns)



   Unnamed: 0                 title      artist        top genre  year  bpm  \
0           1      Hey, Soul Sister       Train       neo mellow  2010   97   
1           2  Love The Way You Lie      Eminem  detroit hip hop  2010   87   
2           3               TiK ToK       Kesha        dance pop  2010  120   
3           4           Bad Romance   Lady Gaga        dance pop  2010  119   
4           5  Just the Way You Are  Bruno Mars              pop  2010  109   

   nrgy  dnce  dB  live  val  dur  acous  spch  pop  
0    89    67  -4     8   80  217     19     4   83  
1    93    75  -5    52   64  263     24    23   82  
2    84    76  -3    29   71  200     10    14   80  
3    92    70  -4     8   71  295      0     4   79  
4    84    64  -5     9   43  221      2     4   78  
Index(['Unnamed: 0', 'title', 'artist', 'top genre', 'year', 'bpm', 'nrgy',
       'dnce', 'dB', 'live', 'val', 'dur', 'acous', 'spch', 'pop'],
      dtype='object')


In [5]:
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

# ✅ Force plotly to render in Jupyter
pio.renderers.default = "iframe_connected"

# --- Spotify Yearly Aggregation ---
spotify_yearly = df_spotify.groupby('year')[['dnce', 'nrgy', 'bpm']].mean().reset_index()
spotify_yearly.rename(columns={
    'year': 'Year',
    'dnce': 'Danceability',
    'nrgy': 'Energy',
    'bpm': 'Tempo'
}, inplace=True)

# --- Merge with S&P 500 Yearly ---
merged = pd.merge(
    sp500_yearly[['Year', 'AvgClose']],
    spotify_yearly,
    on='Year',
    how='inner'
)

# --- Dual Axis Chart ---
fig = go.Figure()

# S&P 500 left axis
fig.add_trace(go.Scatter(
    x=merged['Year'],
    y=merged['AvgClose'],
    name="S&P 500 Avg Close",
    line=dict(color="blue", width=3)
))

# Danceability right axis
fig.add_trace(go.Scatter(
    x=merged['Year'],
    y=merged['Danceability'],
    name="Avg Danceability",
    line=dict(color="orange", width=3),
    yaxis="y2"
))

fig.update_layout(
    title="📈 S&P 500 vs 🎵 Spotify Danceability (2010–2019)",
    xaxis=dict(title="Year"),
    yaxis=dict(title="S&P 500 Avg Close", side="left"),
    yaxis2=dict(
        title="Danceability (0–100)",
        overlaying="y",
        side="right"
    ),
    hovermode="x unified",
    plot_bgcolor="white"
)

fig.show()

# --- Funny Correlation ---
corr = merged['AvgClose'].corr(merged['Danceability'])
print(f"📊 Fake correlation between S&P 500 and danceability: {corr:.2f}")


📊 Fake correlation between S&P 500 and danceability: 0.62


In [6]:
import pandas as pd

f1_champions_points = [
    {"Year": 2010, "Champion": "Sebastian Vettel", "Points": 256},
    {"Year": 2011, "Champion": "Sebastian Vettel", "Points": 392},
    {"Year": 2012, "Champion": "Sebastian Vettel", "Points": 281},
    {"Year": 2013, "Champion": "Sebastian Vettel", "Points": 397},
    {"Year": 2014, "Champion": "Lewis Hamilton",   "Points": 384},
    {"Year": 2015, "Champion": "Lewis Hamilton",   "Points": 381},
    {"Year": 2016, "Champion": "Nico Rosberg",     "Points": 385},
    {"Year": 2017, "Champion": "Lewis Hamilton",   "Points": 363},
    {"Year": 2018, "Champion": "Lewis Hamilton",   "Points": 408},
    {"Year": 2019, "Champion": "Lewis Hamilton",   "Points": 413},
    {"Year": 2020, "Champion": "Lewis Hamilton",   "Points": 347},
    {"Year": 2021, "Champion": "Max Verstappen",   "Points": 395.5},
    {"Year": 2022, "Champion": "Max Verstappen",   "Points": 454},
    {"Year": 2023, "Champion": "Max Verstappen",   "Points": 575},
    {"Year": 2024, "Champion": "Max Verstappen",   "Points": 560}  # example
]

f1_points_df = pd.DataFrame(f1_champions_points)
print(f1_points_df)


    Year          Champion  Points
0   2010  Sebastian Vettel   256.0
1   2011  Sebastian Vettel   392.0
2   2012  Sebastian Vettel   281.0
3   2013  Sebastian Vettel   397.0
4   2014    Lewis Hamilton   384.0
5   2015    Lewis Hamilton   381.0
6   2016      Nico Rosberg   385.0
7   2017    Lewis Hamilton   363.0
8   2018    Lewis Hamilton   408.0
9   2019    Lewis Hamilton   413.0
10  2020    Lewis Hamilton   347.0
11  2021    Max Verstappen   395.5
12  2022    Max Verstappen   454.0
13  2023    Max Verstappen   575.0
14  2024    Max Verstappen   560.0


In [7]:
merged_f1_points = pd.merge(
    sp500_yearly[['Year','AvgClose']],  # from earlier S&P calculation
    f1_points_df,
    on="Year",
    how="inner"
)

print(merged_f1_points)


    Year  AvgClose          Champion  Points
0   2010   1139.97  Sebastian Vettel   256.0
1   2011   1267.64  Sebastian Vettel   392.0
2   2012   1379.35  Sebastian Vettel   281.0
3   2013   1643.80  Sebastian Vettel   397.0
4   2014   1931.38    Lewis Hamilton   384.0
5   2015   2061.07    Lewis Hamilton   381.0
6   2016   2094.65      Nico Rosberg   385.0
7   2017   2449.08    Lewis Hamilton   363.0
8   2018   2746.21    Lewis Hamilton   408.0
9   2019   2913.36    Lewis Hamilton   413.0
10  2020   3217.86    Lewis Hamilton   347.0
11  2021   4273.39    Max Verstappen   395.5
12  2022   4098.51    Max Verstappen   454.0
13  2023   4283.73    Max Verstappen   575.0
14  2024   5428.24    Max Verstappen   560.0


In [8]:
import plotly.graph_objects as go
import plotly.io as pio

pio.renderers.default = "iframe_connected"  # ensures it opens in browser

fig = go.Figure()

# Left axis: S&P 500
fig.add_trace(go.Scatter(
    x=merged_f1_points['Year'],
    y=merged_f1_points['AvgClose'],
    name="S&P 500 Avg Close",
    line=dict(color="blue", width=3)
))

# Right axis: F1 Champion Points
fig.add_trace(go.Scatter(
    x=merged_f1_points['Year'],
    y=merged_f1_points['Points'],
    name="F1 Champion Points",
    line=dict(color="red", width=3),
    yaxis="y2"
))

fig.update_layout(
    title="📈 S&P 500 vs 🏎 F1 Champion Points (2010–2024)",
    xaxis=dict(title="Year"),
    yaxis=dict(title="S&P 500 Avg Close", side="left"),
    yaxis2=dict(
        title="F1 Champion Points",
        overlaying="y",
        side="right"
    ),
    hovermode="x unified",
    plot_bgcolor="white"
)

fig.show()

# Funny correlation
corr_points = merged_f1_points['AvgClose'].corr(merged_f1_points['Points'])
print(f"📊 Fake correlation between S&P 500 and F1 Champion Points: {corr_points:.2f}")


📊 Fake correlation between S&P 500 and F1 Champion Points: 0.78


In [9]:
import pandas as pd

global_temp_data = [
    {"Year": 2010, "TempAnomaly": 0.71},
    {"Year": 2011, "TempAnomaly": 0.62},
    {"Year": 2012, "TempAnomaly": 0.65},
    {"Year": 2013, "TempAnomaly": 0.68},
    {"Year": 2014, "TempAnomaly": 0.74},
    {"Year": 2015, "TempAnomaly": 0.87},
    {"Year": 2016, "TempAnomaly": 0.99},
    {"Year": 2017, "TempAnomaly": 0.91},
    {"Year": 2018, "TempAnomaly": 0.85},
    {"Year": 2019, "TempAnomaly": 0.95},
    {"Year": 2020, "TempAnomaly": 1.02},
    {"Year": 2021, "TempAnomaly": 0.84},
    {"Year": 2022, "TempAnomaly": 0.89},
    {"Year": 2023, "TempAnomaly": 1.12},  # record hot
    {"Year": 2024, "TempAnomaly": 1.05}   # hypothetical
]

temp_df = pd.DataFrame(global_temp_data)
print(temp_df)


    Year  TempAnomaly
0   2010         0.71
1   2011         0.62
2   2012         0.65
3   2013         0.68
4   2014         0.74
5   2015         0.87
6   2016         0.99
7   2017         0.91
8   2018         0.85
9   2019         0.95
10  2020         1.02
11  2021         0.84
12  2022         0.89
13  2023         1.12
14  2024         1.05


In [10]:
merged_temp = pd.merge(
    sp500_yearly[['Year', 'AvgClose']],
    temp_df,
    on="Year",
    how="inner"
)

print(merged_temp)


    Year  AvgClose  TempAnomaly
0   2010   1139.97         0.71
1   2011   1267.64         0.62
2   2012   1379.35         0.65
3   2013   1643.80         0.68
4   2014   1931.38         0.74
5   2015   2061.07         0.87
6   2016   2094.65         0.99
7   2017   2449.08         0.91
8   2018   2746.21         0.85
9   2019   2913.36         0.95
10  2020   3217.86         1.02
11  2021   4273.39         0.84
12  2022   4098.51         0.89
13  2023   4283.73         1.12
14  2024   5428.24         1.05


In [11]:
import plotly.graph_objects as go
import plotly.io as pio

pio.renderers.default = "iframe_connected"  # ensure it opens in browser

fig = go.Figure()

# Left axis: S&P 500
fig.add_trace(go.Scatter(
    x=merged_temp['Year'],
    y=merged_temp['AvgClose'],
    name="S&P 500 Avg Close",
    line=dict(color="blue", width=3)
))

# Right axis: Global Temp Anomaly
fig.add_trace(go.Scatter(
    x=merged_temp['Year'],
    y=merged_temp['TempAnomaly'],
    name="Global Temp Anomaly (°C)",
    line=dict(color="red", width=3),
    yaxis="y2"
))

fig.update_layout(
    title="📈 S&P 500 vs 🌡 Global Temperature Anomaly (2010–2024)",
    xaxis=dict(title="Year"),
    yaxis=dict(title="S&P 500 Avg Close", side="left"),
    yaxis2=dict(
        title="Temperature Anomaly (°C)",
        overlaying="y",
        side="right"
    ),
    hovermode="x unified",
    plot_bgcolor="white"
)

fig.show()

# Fake correlation
corr_temp = merged_temp['AvgClose'].corr(merged_temp['TempAnomaly'])
print(f"📊 Fake correlation between S&P 500 and global temperature: {corr_temp:.2f}")


📊 Fake correlation between S&P 500 and global temperature: 0.75


In [12]:
import pandas as pd

prem_champions_points = [
    {"Year": 2010, "Champion": "Chelsea",           "Points": 86},
    {"Year": 2011, "Champion": "Manchester United", "Points": 80},
    {"Year": 2012, "Champion": "Manchester City",   "Points": 89},
    {"Year": 2013, "Champion": "Manchester United", "Points": 89},
    {"Year": 2014, "Champion": "Manchester City",   "Points": 86},
    {"Year": 2015, "Champion": "Chelsea",           "Points": 87},
    {"Year": 2016, "Champion": "Leicester City",    "Points": 81},
    {"Year": 2017, "Champion": "Chelsea",           "Points": 93},
    {"Year": 2018, "Champion": "Manchester City",   "Points": 100},
    {"Year": 2019, "Champion": "Manchester City",   "Points": 98},
    {"Year": 2020, "Champion": "Liverpool",         "Points": 99},
    {"Year": 2021, "Champion": "Manchester City",   "Points": 86},
    {"Year": 2022, "Champion": "Manchester City",   "Points": 93},
    {"Year": 2023, "Champion": "Manchester City",   "Points": 89},
    {"Year": 2024, "Champion": "Manchester City",   "Points": 91}  # hypothetical
]

prem_df = pd.DataFrame(prem_champions_points)
print(prem_df)


    Year           Champion  Points
0   2010            Chelsea      86
1   2011  Manchester United      80
2   2012    Manchester City      89
3   2013  Manchester United      89
4   2014    Manchester City      86
5   2015            Chelsea      87
6   2016     Leicester City      81
7   2017            Chelsea      93
8   2018    Manchester City     100
9   2019    Manchester City      98
10  2020          Liverpool      99
11  2021    Manchester City      86
12  2022    Manchester City      93
13  2023    Manchester City      89
14  2024    Manchester City      91


In [13]:
merged_prem = pd.merge(
    sp500_yearly[['Year','AvgClose']],  # from earlier S&P calculation
    prem_df,
    on="Year",
    how="inner"
)

print(merged_prem)


    Year  AvgClose           Champion  Points
0   2010   1139.97            Chelsea      86
1   2011   1267.64  Manchester United      80
2   2012   1379.35    Manchester City      89
3   2013   1643.80  Manchester United      89
4   2014   1931.38    Manchester City      86
5   2015   2061.07            Chelsea      87
6   2016   2094.65     Leicester City      81
7   2017   2449.08            Chelsea      93
8   2018   2746.21    Manchester City     100
9   2019   2913.36    Manchester City      98
10  2020   3217.86          Liverpool      99
11  2021   4273.39    Manchester City      86
12  2022   4098.51    Manchester City      93
13  2023   4283.73    Manchester City      89
14  2024   5428.24    Manchester City      91


In [14]:
import plotly.graph_objects as go
import plotly.io as pio

pio.renderers.default = "iframe_connected"  # always open in browser

fig = go.Figure()

# Left axis: S&P 500
fig.add_trace(go.Scatter(
    x=merged_prem['Year'],
    y=merged_prem['AvgClose'],
    name="S&P 500 Avg Close",
    line=dict(color="blue", width=3)
))

# Right axis: PL Champion Points
fig.add_trace(go.Scatter(
    x=merged_prem['Year'],
    y=merged_prem['Points'],
    name="PL Champion Points",
    line=dict(color="green", width=3),
    yaxis="y2"
))

fig.update_layout(
    title="📈 S&P 500 vs ⚽ Premier League Champion Points (2010–2024)",
    xaxis=dict(title="Year"),
    yaxis=dict(title="S&P 500 Avg Close", side="left"),
    yaxis2=dict(
        title="PL Champion Points",
        overlaying="y",
        side="right"
    ),
    hovermode="x unified",
    plot_bgcolor="white"
)

fig.show()

# Funny correlation
corr_prem = merged_prem['AvgClose'].corr(merged_prem['Points'])
print(f"📊 Fake correlation between S&P 500 and PL Champion Points: {corr_prem:.2f}")


📊 Fake correlation between S&P 500 and PL Champion Points: 0.35


In [20]:
# Keep only 2010–2019
combined_df_clean = combined_df[(combined_df['Year'] >= 2010) & (combined_df['Year'] <= 2019)]
combined_df_clean = combined_df_clean.dropna()
print(combined_df_clean)


   Year  AvgClose  Danceability     Energy       Tempo  F1ChampionPoints  \
0  2010   1139.97     64.529412  77.901961  122.058824             256.0   
1  2011   1267.64     63.641509  74.886792  119.075472             392.0   
2  2012   1379.35     65.714286  75.485714  121.085714             281.0   
3  2013   1643.80     62.042254  73.873239  121.676056             397.0   
4  2014   1931.38     62.534483  67.775862  123.000000             384.0   
5  2015   2061.07     63.663158  70.336842  119.768421             381.0   
6  2016   2094.65     63.325000  67.237500  114.325000             385.0   
7  2017   2449.08     65.369231  69.169231  116.800000             363.0   
8  2018   2746.21     67.203125  65.468750  114.593750             408.0   
9  2019   2913.36     69.709677  64.741935  112.451613             413.0   

   TempAnomaly  PLChampionPoints  
0         0.71                86  
1         0.62                80  
2         0.65                89  
3         0.68         

In [23]:
import plotly.graph_objects as go

fig = go.Figure()

# --- S&P 500 ---
fig.add_trace(go.Scatter(
    x=combined_df_clean['Year'],
    y=combined_df_clean['AvgClose'],
    name="📈 S&P 500 Avg Close",
    line=dict(width=3, color="blue")
))

# --- Spotify Danceability ---
fig.add_trace(go.Scatter(
    x=combined_df_clean['Year'],
    y=combined_df_clean['Danceability'],
    name="🎵 Danceability",
    line=dict(width=2, color="orange"),
    yaxis="y2"
))

# --- F1 Champion Points ---
fig.add_trace(go.Scatter(
    x=combined_df_clean['Year'],
    y=combined_df_clean['F1ChampionPoints'],
    name="🏎 F1 Champion Points",
    line=dict(width=2, color="green"),
    yaxis="y3"
))

# --- Global Temp Anomaly ---
fig.add_trace(go.Scatter(
    x=combined_df_clean['Year'],
    y=combined_df_clean['TempAnomaly'],
    name="🌡 Temp Anomaly (°C)",
    line=dict(width=2, color="red"),
    yaxis="y4"
))

# --- Premier League Champion Points ---
fig.add_trace(go.Scatter(
    x=combined_df_clean['Year'],
    y=combined_df_clean['PLChampionPoints'],
    name="⚽ PL Champion Points",
    line=dict(width=2, color="purple"),
    yaxis="y5"
))

# ✅ Corrected layout
fig.update_layout(
    title="Absurd Correlations (2010–2019): S&P 500 vs Music, F1, Temp, & Football",
    xaxis=dict(title="Year"),

    # ✅ Main axis
    yaxis=dict(
        title=dict(text="S&P 500 Avg Close", font=dict(color="blue")),
        tickfont=dict(color="blue")
    ),

    # ✅ Spotify Danceability axis
    yaxis2=dict(
        title=dict(text="Danceability", font=dict(color="orange")),
        tickfont=dict(color="orange"),
        overlaying="y", side="left", position=0.05, anchor="free"
    ),

    # ✅ F1 Points axis
    yaxis3=dict(
        title=dict(text="F1 Points", font=dict(color="green")),
        tickfont=dict(color="green"),
        overlaying="y", side="left", position=0.10, anchor="free"
    ),

    # ✅ Temp axis
    yaxis4=dict(
        title=dict(text="Temp Anomaly °C", font=dict(color="red")),
        tickfont=dict(color="red"),
        overlaying="y", side="right", position=0.95, anchor="free"
    ),

    # ✅ Premier League axis
    yaxis5=dict(
        title=dict(text="PL Points", font=dict(color="purple")),
        tickfont=dict(color="purple"),
        overlaying="y", side="right", position=0.90, anchor="free"
    ),

    plot_bgcolor="white",
    hovermode="x unified",
    legend=dict(orientation="h", y=-0.2)
)

fig.show()


In [24]:
# Features = absurd data, Target = S&P 500 yearly average close
X = combined_df_clean[['Danceability', 'F1ChampionPoints', 'TempAnomaly', 'PLChampionPoints']]
y = combined_df_clean['AvgClose']

from sklearn.model_selection import train_test_split

# 80/20 split (so 8 train years, 2 test years)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Train size:", X_train.shape, "| Test size:", X_test.shape)


Train size: (8, 4) | Test size: (2, 4)


In [25]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

# Train
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Predict
y_pred = lr_model.predict(X_test)

# Evaluate
mae = mean_absolute_error(y_test, y_pred)
print(f"Linear Regression MAE: {mae:.2f}")

# Show actual vs predicted
for true_val, pred_val in zip(y_test, y_pred):
    print(f"True: {true_val:.2f} → Predicted: {pred_val:.2f}")


Linear Regression MAE: 27.60
True: 2746.21 → Predicted: 2734.17
True: 1267.64 → Predicted: 1310.81


In [26]:
coef_df = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': lr_model.coef_
}).sort_values(by='Coefficient', key=abs, ascending=False)

print("\nFeature influence on S&P 500 (completely absurd):")
print(coef_df)



Feature influence on S&P 500 (completely absurd):
            Feature  Coefficient
2       TempAnomaly  2191.563898
3  PLChampionPoints    39.082982
0      Danceability    20.649050
1  F1ChampionPoints     4.006195


In [27]:
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Normalize
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Simple NN
nn_model = keras.Sequential([
    layers.Input(shape=(X_train.shape[1],)),
    layers.Dense(8, activation='relu'),
    layers.Dense(4, activation='relu'),
    layers.Dense(1)
])

nn_model.compile(optimizer='adam', loss='mse', metrics=['mae'])
nn_model.fit(X_train_scaled, y_train, epochs=200, verbose=0)

loss, mae_nn = nn_model.evaluate(X_test_scaled, y_test, verbose=0)
print(f"\nNeural Net MAE: {mae_nn:.2f}")

y_pred_nn = nn_model.predict(X_test_scaled).flatten()
for true_val, pred_val in zip(y_test, y_pred_nn):
    print(f"[NN] True: {true_val:.2f} → Predicted: {pred_val:.2f}")



Neural Net MAE: 2004.96
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 321ms/step
[NN] True: 2746.21 → Predicted: 3.17
[NN] True: 1267.64 → Predicted: 0.77
